# Processing OISST Data with Python 3.6 & ERDDAPY

**NOTE**

ERDDAPY does not currently support generating dowload url's for griddap data. i.e. fails for OISST

In [2]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import erddapy 
import netCDF4
import os
os.chdir("/Users/akemberling/Documents/oisst_mainstays/annual_oisst")

## 1. Instantiate erdappy server object:

In [8]:
from erddapy import ERDDAP


e = ERDDAP(
    #server='https://data.ioos.us/gliders/erddap',
    server='https://coastwatch.pfeg.noaa.gov/erddap',
    protocol='griddap',
    response='nc',
)

## 2. Add constraints 
Now we can populate the object with constraints, the variables of interest, and the dataset id.

In [12]:
e.dataset_id = 'ncdcOisst2Agg'

e.constraints = {
    #time range
    'time>=': '1982-01-01',
    'time<=': '1985-12-31',
    
    #Full Extent
    #'latitude>=': -89.875,
    #'latitude<=': 89.875,
    #'longitude>=': 0.125,
    #'longitude<=': 359.875,
    
    #Gulf of Maine
    #c(-77, -60, 35, 46)
    'latitude>=': 35,
    'latitude<=': 46,
    'longitude>=': 103,
    'longitude<=': 120,
}

e.variables = [
    'sst',
    'lat',
    'lon',
    'time',
]


download_url = e.get_download_url()

print(download_url)

HTTPError: 400 Client Error:  for url: https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2Agg.nc?sst,lat,lon,time&time%3E=378691200.0&time%3C=504835200.0&latitude%3E=35&latitude%3C=46&longitude%3E=103&longitude%3C=120

In [10]:
'''https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2Agg.nc
?sst,lat,lon,time&time%3E=378691200.0&time%3C=504835200.0&latitude%3E=35&latitude%3C=46&longitude%3E=103&longitude%3C=120'''

'''https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2Agg.nc
?sst
[(1982-01-01):1:(1985-12-31)][(0.0):1:(0.0)][(35):1:(46)][(103):1:(120)]'''

'https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst2Agg.nc?sst[(1982-01-01):1:(1985-12-31)][(0.0):1:(0.0)][(35):1:(46)][(103):1:(120)]'

## 3. Access Netcdf array data with xarray

The link we generate will download the grid data as a temporary file

In [4]:
import requests

def humansize(nbytes):
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    k = 0
    while nbytes >= 1024 and k < len(suffixes)-1:
        nbytes /= 1024.
        k += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[k])

r = requests.head(download_url)
nbytes = float(r.headers['Content-Length'])
humansize(nbytes)

'169 B'

In [98]:
r.headers['Content-Type']

'text/html'

## Download as an array object - Not Working

In [5]:
ds = e.to_xarray(decode_times = True)

ds

OSError: [Errno -51] NetCDF: Unknown file format: b'/var/folders/bt/wcx80dtj3hs31w2qt03shxg00000gn/T/erddapy_rmozfja6.nc'